In [1]:
import drms
import os
import numpy as np
import math
from tqdm import tqdm

In [2]:
series = 'hmi.sharp_cea_720s'
segments = ['Bt','Br','Bp']
kwlist = ['T_REC','HARPNUM','USFLUX','LON_FWT','LAT_FWT','NOAA_ARS','AREA']

c = drms.Client(email = 'example@email.com', verbose=False)
out_dir = 'C:\\sharp_data_v3'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [5]:
download_array = [] # range(7050,8369)
for sharpnum in tqdm(range(8369,9678)):
    k = c.query('%s[%d]' % (series, sharpnum), key=kwlist)
    try:
        k = k.loc[k.LON_FWT.abs() < 60]
        k = k.loc[k.LAT_FWT.abs() < 60]
        k = k.loc[k.AREA > 100] # 100 MSH is approx. 100x100 *10^5km, so approx. 100x100 pixels.
        k = k.loc[k.USFLUX > k.USFLUX.quantile(0.75)]
        rec_sample = (k.LON_FWT.pow(2)+k.LAT_FWT.pow(2)).idxmin()
    except AttributeError:
        # Sharp number not present
        continue
    except ValueError:
        # Sharp too close to disk edge
        continue
#     print('{number}{ar}: {flux} @ ({lat},{lon})'.format(
#         number=sharpnum, ar = (f' ({k.NOAA_ARS[rec_sample]})' if k.NOAA_ARS[rec_sample] != 'MISSING' else ''),
#         flux=k.USFLUX[rec_sample], lat=k.LAT_FWT[rec_sample], lon=k.LON_FWT[rec_sample]
#     ))
    k_sample = k.loc[rec_sample]
    t_sample = datetime(k.T_REC[rec_sample])
    fname_mask = '{series}[{sharpnum}][{tstr}]'
    download_array.append(fname_mask.format(
        series=series,sharpnum=sharpnum,tstr=t_sample.queryFormat()
    ) + '{%s}'%(','.join(segments)))


100%|██████████████████████████████████████████████████████████████████████████████| 1309/1309 [32:51<00:00,  1.51s/it]


In [6]:
# JSOC gets mad if you request too many files at once, so we have to split up into smaller requests.
arrs = np.array_split(download_array,math.ceil(len(download_array)/15))

for arr in tqdm(arrs):
    exp_query = ','.join(arr)
#     print('Request: ' + exp_query)
    request = c.export(exp_query)
    request.download(out_dir)

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [44:11<00:00, 58.91s/it]


In [4]:
class datetime:
    def __init__(self, str):
        arr = str.split('_')
        self.date = arr[0].split('.')
        self.time = arr[1].split(':')
        self.TAI = (arr[2] == 'TAI')
        self.year = self.date[0]
        self.month = self.date[1]
        self.day = self.date[2]
        self.hour = self.time[0]
        self.minute = self.time[1]
        self.second = self.time[2]
    def sharpFormat(self):
        string = '{year}{month}{day}_{hour}{minute}{second}'
        if self.TAI:
            string = string + '_TAI'
        return string.format(
            year=self.year,month=self.month,day=self.day,
            hour=self.hour,minute=self.minute,second=self.second
        )
    def queryFormat(self):
        string = '{year}.{month}.{day}_{hour}:{minute}:{second}'
        return string.format(
            year=self.year,month=self.month,day=self.day,
            hour=self.hour,minute=self.minute,second=self.second
        )

In [ ]:
k = c.query('%s[%d]' % (series, 7300), key=kwlist)

In [ ]:
k.AREA